In [24]:
import duckdb

In [25]:
# raw .csv filename
csv_filename = "Tesla-2024-03-10 13-34-28-2024-03-10 14-25-11.csv"
# duckdb filename (to create)
db_filename = "file.db"
# name of view (to create)
data_view_name = "data_v"

In [26]:
# query to create a view that aliases each column name in the .csv to
# something more SQL query friendly
create_data_view_query = f"""create or replace view {data_view_name} as
                 select "Timestamp (PDT)" as ts,
                 "State" as state,
                 "Charging State" as charging_state,
                 "Shift State" as shift_state,
                 "Battery Level (%)" as battery_level_pct,
                 "Usable Battery Level (%)" as usable_battery_level_pct,
                 "Battery Range (mi)" as battery_range_mi,
                 "Est. Battery Range (mi)" as estimated_battery_range_mi,
                 "Ideal Battery Range (mi)" as ideal_battery_range_mi,
                 "Latitude" as latitude,
                 "Longitude" as longitude,
                 "Elevation (m)" as elevation_m,
                 "Speed (mph)" as speed_mph,
                 "Power (kW)" as power_kw,
                 "Odometer (mi)" as odometer_mi,
                 "Charge Rate (mph)" as charge_rate_mph,
                 "Current (A)" as current_a,
                 "Charger Power (kW)" as charger_power_kw,
                 "Charger Voltage (V)" as charger_voltage_v,
                 "Charger Phases" as charger_phases,
                 "Energy Added (kWh)" as energy_added_kwh,
                 "Rated Range Added (mi)" as rated_range_added_mi,
                 "Ideal Range Added (mi)" as ideal_range_added_mi,
                 "Climate" as climate,
                 "Battery Heater" as battery_heater,
                 "Inside Temp (°F)" as inside_temperature_f,
                 "Outside Temp (°F)" as outside_temperature_f,
                 "Locked" as locked,
                 "Sentry Mode" as sentry_mode
                 from '{csv_filename}'"""

In [27]:
def check_view_exists(duck_conn: duckdb.DuckDBPyConnection, check_view_name: str) -> bool:
    """Return True if check_view_name exists in duck_conn or False if not"""
    try:
        # use DuckDB Python API, not SQL query
        duck_conn.view(check_view_name)
    except Exception as local_e:
        if f"Table with name {check_view_name} does not exist" in str(local_e):
            return False
    return True

In [28]:
def create_view(duck_conn: duckdb.DuckDBPyConnection, duck_filename: str, new_view_query: str) -> duckdb.DuckDBPyConnection:
    """Open database connection to duck_filename in read / write mode,
    run the new_view_query SQL query,
    re-open the database connection in read only mode,
    and return the read only DuckDB connection"""

    duck_conn.close()

    duck_conn = duckdb.connect(duck_filename, read_only=False)
    duck_conn.query(new_view_query)
    duck_conn.close()

    duck_conn = duckdb.connect(duck_filename, read_only=True)

    return duck_conn

In [29]:
# connect to db in read mode
# if db does not exist, create it in write mode and reconnect in read mode

# close connection if it exists
if "conn" in globals():
    # noinspection PyUnresolvedReferences
    conn.close()
try:
    conn = duckdb.connect(db_filename, read_only=True)

except Exception as e:
    if "database does not exist" in str(e):
        conn = duckdb.connect(db_filename, read_only=False)
        conn.close()
        conn = duckdb.connect(db_filename, read_only=True)
    else:
        raise e

In [30]:
# see if our view already exists in the db, if view does not exist, create it

view_exists = check_view_exists(conn, data_view_name)

if not view_exists:
    conn = create_view(conn, db_filename, create_data_view_query)

In [31]:
# confirm that view exists, if not assert because something went wrong
view_exists = check_view_exists(conn, data_view_name)

assert view_exists, "Error: view not created"

Now we need to configure this DuckDB database as a Data Source in PyCharm. Here's the [PyCharm documentation](https://www.jetbrains.com/help/pycharm/databases.html) for the Database tool and here's a step by step for doing configuring it to target DuckDB:

1. Click the *Database* icon on the right
2. Click the *+* icon in the Database menu
3. Click *Data Source*
4. Scroll to the bottom and select *Other*
5. Select *DuckDB*

![Setup steps 1 - 5](img/setup_1.png)

6. Set *File* to the value of the `db_filename` variable in Python (`file.db` in our case)
7. Set *Authentication* to *No auth*
8. Click *Test Connection*
    1. You may be prompted to install database drivers for DuckDB
9. Click on the *Schemas* tab

![Setup steps 6 - 9](img/setup_2.png)

10. Open the `file` expander
11. Check the box for `main` under `file`
12. Click the *Advanced* tab

![Setup steps 10 - 12](img/setup_3.png)

13. Add a setting with *Name* `duckdb.read_only`
14. Set the *Value* to `true`
15. Click *OK*

![Setup steps 13 - 15](img/setup_4.png)

16. We can now see introspection of the .csv file in the Database window!
17. The *Deactivate* button disconnects PyCharm from the database 
    1. **Note:** Any time you connect to `db_filename` in read / write mode in Python, you will need to click *Deactivate*
    2. Similarly, to disconnect from the database in Python, use the method: `conn.close()`
18. The *Refresh* button connects PyCharm to the database

![Setup steps 16 - 18](img/setup_5.png)

Concurrency in DuckDB can be either:

1. Multiple *read only* connections to a database
2. A **single** *read / write* connection to a database
    1. A *read only* connection cannot be made when there is an active *read / write* connection

We can now use PyCharm's syntax highlighting and autocompletion features when we query DuckDB. Here's a template to start with:

```python
conn = duckdb.connect(db_filename, read_only=True)
conn.query(f"""
select 
from data_v -- data_v cannot be replaced with a Python variable
""").df()
```

Here's what it looks like in action where I've just typed `l` and let the autocomplete engine show me what columns I can choose from. Awesome!

![autocomplete example](img/autocomplete.png)

In [32]:
# here's a demo query for the sake of completeness
# first we open a read only connection to the database
# we select the elevation rounded to the nearest 100 m
# we aggregate the outside temperature in °F by each 100 m elevation bin
# we order the results by rounded elevation, descending to be a little more intuitive with the downhill drive
# and finally we close the connection to the database

conn = duckdb.connect(db_filename, read_only=True)

print(conn.query(f"""
select round(elevation_m / 100.0) * 100 as round_elevation_m,
avg(outside_temperature_f) as avg_outside_temperature_f
from data_v
group by round_elevation_m
order by round_elevation_m desc
""").df())

conn.close()

    round_elevation_m  avg_outside_temperature_f
0              2300.0                  37.000000
1              2100.0                  37.000000
2              2000.0                  37.333333
3              1700.0                  40.000000
4              1600.0                  40.000000
5              1500.0                  41.818182
6              1400.0                  43.000000
7              1300.0                  45.375000
8              1200.0                  47.333333
9              1100.0                  48.666667
10             1000.0                  51.000000
11              900.0                  51.000000
12              800.0                  51.000000
13              700.0                  51.444444
14              600.0                  53.500000
15              500.0                  54.800000
16              400.0                  55.000000
